<a href="https://colab.research.google.com/github/mayaonair/Lansia/blob/main/Summary_dan_Classify_Lansia_2045.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community requests==2.32.4
!pip install replicate

In [ ]:
from langchain_community.llms import Replicate
import os
from google.colab import userdata

# Set the API token
api_token = userdata.get('api_token')
os.environ["REPLICATE_API_TOKEN"] = api_token

# Model setup
model = "ibm-granite/granite-3.3-8b-instruct"
output = Replicate(
model=model,
replicate_api_token=api_token,
)

Langkah 1: Import file csv

In [ ]:
import pandas as pd

file_path = '/content/Kumpulan Artikel Lansia 2045(Sheet1).csv'

try:
    artikel_df = pd.read_csv(file_path, sep=';', encoding='latin-1')
    print("Data artikel berhasil dimuat:")
    display(artikel_df.head())


except FileNotFoundError:
    print(f"Error: File not found at {file_path}. Please check the file path and name.")
except Exception as e:
    print(f"An error occurred: {e}")

Data artikel berhasil dimuat dengan pemisah titik koma:


,No,Sumber,Tanggal,Judul,Artikel
0,NaN,https://www.dbs.com/newsroom/Lansia_Indonesia_...,11/06/2025,"Lansia Indonesia tembus 20% pada 2045, DBS Fou...","Indonesia, 12 Jun 2025 - Indonesia tengah meng..."
1,NaN,https://www.kompas.id/artikel/lonjakan-pendudu...,09/05/2025,Lonjakan Penduduk Lansia Indonesia dan Ancaman...,Jumlah penduduk senior terus naik dan diperkir...
2,NaN,https://mediaindonesia.com/humaniora/778494/pa...,01/06/2025,"Pada 2045, Ada 65,82 juta jiwa Lansia di Indon...","SAAT ini, Indonesia tengah menghadapi tantanga..."
3,NaN,https://kumparan.com/kumparannews/indonesia-em...,01/02/2025,Indonesia Emas 2045: Siapkah Kita dengan Ledak...,Indonesia kini mengalami ageing population ata...
4,NaN,https://www.kanopifebui.com/post/pasca-2045-si...,04/09/2025,Pasca 2045: Siapkah Indonesia untuk Era Popula...,Pembukaan \n\nIndonesia menapaki tahun 2045 se...


Langkah 2: Summarization per article -> definisiin parameternya > karena panjang artikel variatif maka dibikin biar maxs tokennya nanti menyesuaikan sama panjang artikel.

In [ ]:
# Define base parameter (max_tokens akan disesuaikan per artikel)
base_generation_parameters = {
    "top_p": 0.9,
    "top_k": 10,
    "repetition_penalty": 1.1,
    # "stopping_sequence": ["\n\n"]
}

# Parameter max_tokens disesuaikan berdasar panjang artikel
BASE_MAX_TOKENS = 150 # Minimum max_tokens
MAX_TOKENS_PER_CHAR = 0.2 # Increased tokens per character significantly
MAX_POSSIBLE_TOKENS = 1000 # Increased the upper limit for max_tokens


# List baru
model_summaries = []

# Loop melalui setiap baris (artikel) di DataFrame
for index, row in artikel_df.iterrows():
    artikel_text = row['Artikel']

    # Calculate adjusted max_tokens based on article length
    # Using character count as a simple proxy for length
    article_length = len(artikel_text)
    adjusted_max_tokens = BASE_MAX_TOKENS + int(article_length * MAX_TOKENS_PER_CHAR)

    # Ensure adjusted_max_tokens does not exceed a reasonable upper limit
    # (e.g., based on model capabilities or desired max summary length)
    adjusted_max_tokens = min(adjusted_max_tokens, MAX_POSSIBLE_TOKENS)


    # Combine base parameters with adjusted max_tokens
    current_generation_parameters = base_generation_parameters.copy()
    current_generation_parameters["max_tokens"] = adjusted_max_tokens


    # Prompt untuk summarization per artikel dalam Bahasa Indonesia
    prompt = f"""
    Ringkas artikel berikut dalam Bahasa Indonesia, fokus pada **temuan utama, argumen kunci, atau kesimpulan** yang paling penting. Sajikan ringkasan dalam **satu paragraf singkat, terdiri dari 1 hingga 3 kalimat** baru. Pastikan ringkasan hanya berisi informasi esensial dan relevan dengan topik utama lansia.

    Artikel:
    {artikel_text}
    """

    # Invoke the model with the prompt for the current article and adjusted parameters
    try:
        response = output.invoke(prompt, **current_generation_parameters)
    except Exception as e:
        print(f"Error invoking model for article {row['Judul']}: {e}")
        response = "Gagal merangkum artikel." # Assign a default message on error


    # Append the model's response to the list
    model_summaries.append(response)

# Tambahkan kolom 'Ringkasan_Model' ke DataFrame dengan hasil dari model
artikel_df['Ringkasan_Model'] = model_summaries

print("\n--- DataFrame dengan Model Summaries ---")
display(artikel_df[['Judul', 'Ringkasan_Model']])

NameError: name 'artikel_df' is not defined

Langkah 3: Klasifikasiin kategori per artikel

In [ ]:
#lists to store the classification results and explanations
classification_categories = []
classification_explanations = []

# Loop through each row (article) in the DataFrame
for index, row in artikel_df.iterrows():
    artikel_text = row['Ringkasan_Model']

    # Prompt for classification per article
    prompt = f"""
    Klasifikasikan artikel berikut ke dalam satu atau lebih kategori berikut yang paling relevan:
    - Kesehatan Lansia
    - Kebijakan Pemerintah
    - Inovasi Teknologi
    - Sosial Budaya
    - Ekonomi
    - Lainnya

    Setelah mengklasifikasikan, **PASTIKAN untuk memberikan penjelasan singkat mengapa artikel tersebut masuk ke dalam kategori tersebut.**

    Artikel:
    {artikel_text}

    Format Output:
    Kategori: [Kategori 1], [Kategori 2], ...
    Penjelasan: [Penjelasan singkat]
    """

    # Invoke the model with the prompt for the current article
    response = output.invoke(prompt)

    # Process the model's response to separate categories and explanation
    categories = "Tidak Diketahui"  # Default value
    explanation = "Tidak ada penjelasan" # Default value

    # Split the response by lines and look for "Kategori:" and "Penjelasan:"
    lines = response.strip().split('\n')
    for line in lines:
        if line.startswith("Kategori:"):
            categories = line.replace("Kategori:", "").strip()
        elif line.startswith("Penjelasan:"):
            explanation = line.replace("Penjelasan:", "").strip()

    classification_categories.append(categories)
    classification_explanations.append(explanation)


# Add new columns 'Klasifikasi_Kategori' and 'Klasifikasi_Penjelasan' to the DataFrame
artikel_df['Klasifikasi_Kategori'] = classification_categories
artikel_df['Klasifikasi_Penjelasan'] = classification_explanations

print("\n--- DataFrame with Model Classification ---")
display(artikel_df[['Judul', 'Klasifikasi_Kategori', 'Klasifikasi_Penjelasan']])


--- DataFrame with Model Classification ---


,Judul,Klasifikasi_Kategori,Klasifikasi_Penjelasan
0,"Lansia Indonesia tembus 20% pada 2045, DBS Fou...","Kesehatan Lansia, Kebijakan Pemerintah, Ekonomi",Artikel ini diskusikan tentang pergeseran demo...
1,Lonjakan Penduduk Lansia Indonesia dan Ancaman...,"Kesehatan Lansia, Kebijakan Pemerintah, Ekonomi",Artikel ini fokus pada tingkat tua populasi In...
2,"Pada 2045, Ada 65,82 juta jiwa Lansia di Indon...","Kesehatan Lansia, Kebijakan Pemerintah",Artikel ini fokus pada tantangan lansia di Ind...
3,Indonesia Emas 2045: Siapkah Kita dengan Ledak...,"Kesehatan Lansia, Ekonomi",Artikel ini diskusikan tentang perubahan struk...
4,Pasca 2045: Siapkah Indonesia untuk Era Popula...,"Kesehatan Lansia, Ekonomi, Kebijakan Pemerintah",Artikel ini diskusikan tentang tantangan dan p...
5,"Lansia Indonesia Tembus 20% pada 2045, Bagaima...","Kesehatan Lansia, Ekonomi",Artikel ini fokus pada perubahan demografis In...
6,Hak Lansia di Era Digital,"Kesehatan Lansia, Kebijakan Pemerintah, Sosial...",Artikel ini terkait dengan persiapan Indonesia...
7,Menyiapkan Lansia Mandiri dan Sehat Holistik M...,"Kesehatan Lansia, Kebijakan Pemerintah",Artikel ini diskusi tentang respon pemerintah ...
8,Beri Peluang Lansia Berkontribusi Capai Visi I...,"Kebijakan Pemerintah, Kesehatan Lansia",Artikel ini diskusikan tentang pertemuan antar...
9,Indonesia Siapkan Lansia Aktif dan Produktif,Kesehatan Lansia,Artikel ini fokus pada populasi lansia di Indo...


In [ ]:
# Display the DataFrame
print("--- Tabel Ringkasan dan Klasifikasi Artikel ---")
display(artikel_df[['Judul', 'Ringkasan_Model', 'Klasifikasi_Kategori', 'Klasifikasi_Penjelasan']])

Langkah 4: Identifikasi dan klasifikasi stakeholder ke dalam beberapa kriteria

In [ ]:
import pandas as pd
import re
from collections import defaultdict

file_path = '/content/Kumpulan Artikel Lansia 2045(Sheet1).csv'

# Dict kata kunci untuk mengidentifikasi pemangku kepentingan
# dan informasi terkait (fokus dan inisiatif)
stakeholder_data = {
    'Pemerintah Pusat': {
        'keywords': ['Kementerian Kesehatan', 'Kemenkes', 'Bappenas', 'Kementerian PPN', 'Kemenko PMK', 'Kementerian Sosial', 'Kemensos', 'Kementerian PUPR', 'BKKBN', 'KemenPPPA', 'Kementerian Pemberdayaan Perempuan dan Perlindungan Anak'],
        'fokus': 'Pembuatan kebijakan dan program skala nasional, alokasi anggaran, dan koordinasi lintas sektor.',
        'inisiatif': {
            'Kementerian Kesehatan': ['Menerapkan program deteksi dini', 'Mengadakan Cek Kesehatan Gratis (CKG)', 'Menyelenggarakan edukasi kesehatan dan gaya hidup sehat (CERDIK)'],
            'Bappenas': ['Mengintegrasikan silver economy ke RPJPN 2025-2045', 'Menginisiasi program Layanan Lansia Terintegrasi (LLT)', 'Mendorong perawatan jangka panjang (long-term care)'],
            'Kemenko PMK': ['Mengubah paradigma terkait bantuan sosial', 'Menyoroti potensi silver economy', 'Mendorong layanan kependudukan "jemput bola"'],
            'Kemensos': ['Memastikan negara hadir melalui program sosial', 'Melaksanakan Program Asistensi Rehabilitasi Sosial (ATENSI)'],
            'Kementerian PUPR': ['Membangun rusun khusus lansia', 'Merancang hunian yang terintegrasi dengan layanan kesehatan'],
            'BKKBN': ['Mengembangkan program Bina Keluarga Lansia (BKL)', 'Meluncurkan aplikasi GoLantang dan Sekolah Lansia']
        }
    },
    'Pemerintah Daerah': {
        'keywords': ['Pemerintah Daerah Istimewa Yogyakarta', 'Pemda DIY', 'Pemkot Cimahi', 'Pemerintah Kota Balikpapan', 'Dinas Kesehatan'],
        'fokus': 'Implementasi kebijakan di tingkat lokal, pembangunan infrastruktur, dan layanan publik.',
        'inisiatif': {
            'Pemerintah Daerah Istimewa Yogyakarta': ['Menerbitkan Perda No. 3 Tahun 2021', 'Menyelenggarakan seminar dan lokakarya untuk kolaborasi'],
            'Pemerintah Kota Balikpapan': ['Menginisiasi Rancangan Peraturan Daerah (Raperda) Ramah Lansia'],
            'Pemerintah Kota Cimahi': ['Merancang pembangunan Taman Lansia', 'Menyelenggarakan layanan kesehatan rutin']
        }
    },
    'Sektor Swasta': {
        'keywords': ['DBS Foundation', 'Bank DBS Indonesia', 'Living Well Seniors Communities', 'Mayapada Hospital', 'PT Blue Bird Tbk'],
        'fokus': 'Mengembangkan layanan dan produk finansial, kesehatan, dan ekonomi yang berorientasi pada lansia (silver economy).',
        'inisiatif': {
            'Bank DBS Indonesia': ['Melakukan survei DBS Ageing Society', 'Mengadakan diskusi Impact Beyond Dialogue', 'Mengembangkan solusi perbankan untuk perencanaan pensiun'],
            'PT Blue Bird Tbk': ['Meluncurkan layanan Lifecare Taxi'],
            'Living Well Seniors Communities': ['Berbagi pengalaman dari negara lain', 'Mendorong investasi infrastruktur perawatan'],
            'Mayapada Hospital': ['Memberikan pelayanan kesehatan dan skrining penyakit']
        }
    },
    'Akademisi & Peneliti': {
        'keywords': ['Universitas Indonesia', 'UI', 'Telkom University', 'Universitas Ahmad Dahlan', 'UAD', 'BRIN', 'Badan Riset dan Inovasi Nasional', 'The SMERU Institute', 'Pakar'],
        'fokus': 'Penelitian, analisis kebijakan, dan pengembangan inovasi berbasis teknologi dan sosial.',
        'inisiatif': {
            'Universitas Indonesia': ['Meneliti masalah kesehatan dan mental lansia', 'Mengembangkan aplikasi e-Health Care Pandu-Ina'],
            'Telkom University': ['Menciptakan inovasi De-Lansia'],
            'Universitas Ahmad Dahlan': ['Bekerja sama dengan Social Club Indonesia untuk mencegah demensia'],
            'BRIN': ['Meneliti efektivitas program perlindungan sosial']
        }
    },
    'Organisasi Masyarakat Sipil & Komunitas': {
        'keywords': ['Alzheimer Indonesia', 'Social Club Indonesia', 'SCI', 'Aisyiyah', 'Yayasan Karya Bhakti RIA Pembangunan'],
        'fokus': 'Pemberdayaan, advokasi, dan penyediaan layanan sosial langsung di tingkat komunitas.',
        'inisiatif': {
            'Alzheimer Indonesia': ['Menekankan pentingnya pemberdayaan lansia'],
            'Aisyiyah': ['Mengelola panti dan daycare lansia'],
            'Social Club Indonesia': ['Menyelenggarakan kegiatan sosial dan fisik'],
            'Yayasan Karya Bhakti RIA Pembangunan': ['Mengelola rusun lansia']
        }
    }
}

try:
    df_artikel = pd.read_csv(file_path, sep=';', encoding='latin-1')

    # Dict untuk menyimpan pemangku kepentingan unik dengan detailnya
    unique_stakeholders = {}

    # Iterasi melalui setiap baris DataFrame
    for _, row in df_artikel.iterrows():
        artikel_text = str(row['Artikel'])

        for category, data in stakeholder_data.items():
            for keyword in data['keywords']:
                if re.search(r'\b' + re.escape(keyword) + r'\b', artikel_text, re.IGNORECASE):
                    # Identifikasi nama standar
                    standard_name = keyword
                    for name_variant, standard_name_key in data['inisiatif'].items():
                        if name_variant.lower() in keyword.lower():
                            standard_name = name_variant
                            break

                    # Tambahin ke dictionary unik jika belum ada
                    if standard_name not in unique_stakeholders:
                        initiatives = data['inisiatif'].get(standard_name, [])
                        initiatives_str = '\n- '.join(initiatives) if initiatives else 'Tidak disebutkan secara spesifik.'

                        unique_stakeholders[standard_name] = {
                            'Kategori Organisasi': category,
                            'Fokus Utama': data['fokus'],
                            'Rencana/Inisiatif yang Dilakukan': '- ' + initiatives_str
                        }

    # Ubah dict ke DataFrame
    df_stakeholder = pd.DataFrame.from_dict(unique_stakeholders, orient='index')
    df_stakeholder.index.name = 'Pemangku Kepentingan'
    df_stakeholder.reset_index(inplace=True)

    pd.set_option('display.max_colwidth', None)

    print("Tabel Klasifikasi Pemangku Kepentingan Berdasarkan Analisis Data Anda:\n")
    display(df_stakeholder)

except FileNotFoundError:
    print(f"Error: File not found at {file_path}. Please check the file path and name.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd
import re
from collections import defaultdict

# Tentukan path ke file CSV Anda. Ganti path ini jika lokasinya berbeda.
file_path = '/content/Kumpulan Artikel Lansia 2045(Sheet1).csv'

# Kamus data pemangku kepentingan yang kini menyertakan fokus masalah
stakeholder_data = {
    'Pemerintah Pusat': {
        'keywords': ['Kementerian Kesehatan', 'Kemenkes', 'Bappenas', 'Kementerian PPN', 'Kemenko PMK', 'Kementerian Sosial', 'Kemensos', 'Kementerian PUPR', 'BKKBN', 'KemenPPPA', 'Kementerian Pemberdayaan Perempuan dan Perlindungan Anak'],
        'data': {
            'Kementerian Kesehatan': {
                'fokus': 'Peningkatan kesehatan dan kualitas hidup lansia agar tetap aktif dan produktif.',
                'inisiatif': ['Menerapkan program deteksi dini', 'Mengadakan Cek Kesehatan Gratis (CKG)', 'Menyelenggarakan edukasi kesehatan dan gaya hidup sehat (CERDIK)'],
                'fokus_masalah': ['Rendahnya UHH sehat', 'Lansia rentan terhadap penyakit kronis', 'Kurangnya skrining dini']
            },
            'Bappenas': {
                'fokus': 'Perencanaan kebijakan jangka panjang untuk transisi demografi dan pengelolaan aging population.',
                'inisiatif': ['Mengintegrasikan silver economy ke RPJPN 2025-2045', 'Menginisiasi program Layanan Lansia Terintegrasi (LLT)'],
                'fokus_masalah': ['Ancaman kegagalan bonus demografi kedua', 'Kesenjangan layanan terintegrasi', 'Belum optimalnya strategi nasional']
            },
            'Kemenko PMK': {
                'fokus': 'Kesejahteraan sosial dan penanggulangan kemiskinan ekstrem pada kelompok rentan, termasuk lansia.',
                'inisiatif': ['Mengubah paradigma terkait bantuan sosial', 'Menyoroti potensi silver economy', 'Mendorong layanan kependudukan "jemput bola"'],
                'fokus_masalah': ['Kesenjangan dalam distribusi bantuan sosial', 'Kesulitan akses administrasi kependudukan bagi lansia']
            },
            'Kemensos': {
                'fokus': 'Perlindungan sosial dan pemenuhan kebutuhan dasar lansia.',
                'inisiatif': ['Memastikan negara hadir melalui program sosial', 'Melaksanakan Program Asistensi Rehabilitasi Sosial (ATENSI)'],
                'fokus_masalah': ['Tingginya jumlah lansia terlantar', 'Keterbatasan kapasitas panti jompo yang dikelola pemerintah']
            },
            'Kementerian PUPR': {
                'fokus': 'Penyediaan hunian dan infrastruktur yang layak, aman, dan nyaman bagi lansia.',
                'inisiatif': ['Membangun rusun khusus lansia', 'Merancang hunian yang terintegrasi dengan layanan kesehatan'],
                'fokus_masalah': ['Keterbatasan hunian layak dan terjangkau', 'Kurangnya variasi tipe hunian selain panti jompo']
            },
            'BKKBN': {
                'fokus': 'Pengelolaan populasi dan pemberdayaan lansia.',
                'inisiatif': ['Mengembangkan program Bina Keluarga Lansia (BKL)', 'Meluncurkan aplikasi GoLantang dan Sekolah Lansia'],
                'fokus_masalah': ['Populasi lansia yang terus meningkat dan tidak produktif', 'Rendahnya partisipasi lansia dalam program pemberdayaan']
            }
        }
    },
    'Pemerintah Daerah': {
        'keywords': ['Pemerintah Daerah Istimewa Yogyakarta', 'Pemda DIY', 'Pemkot Cimahi', 'Pemerintah Kota Balikpapan', 'Dinas Kesehatan'],
        'data': {
            'Pemerintah Daerah Istimewa Yogyakarta': {
                'fokus': 'Mewujudkan DIY sebagai kawasan ramah lansia melalui regulasi dan kolaborasi.',
                'inisiatif': ['Menerbitkan Perda No. 3 Tahun 2021 dan Pergub No. 4 Tahun 2024', 'Menyelenggarakan seminar dan lokakarya untuk kolaborasi'],
                'fokus_masalah': ['Keterbatasan kolaborasi lintas sektor', 'Tingginya persentase lansia (17,4%) yang membutuhkan layanan inklusif']
            },
            'Pemerintah Kota Balikpapan': {
                'fokus': 'Mendorong kesejahteraan dan produktivitas lansia di tingkat kota.',
                'inisiatif': ['Menginisiasi Rancangan Peraturan Daerah (Raperda) Ramah Lansia'],
                'fokus_masalah': ['Lonjakan populasi lansia', 'Penuaan penduduk yang dapat memberi dampak negatif pada kemampuan fiskal']
            },
            'Pemerintah Kota Cimahi': {
                'fokus': 'Peningkatan kualitas hidup lansia dan penyediaan ruang publik yang inklusif.',
                'inisiatif': ['Merancang pembangunan Taman Lansia', 'Menyelenggarakan layanan kesehatan rutin'],
                'fokus_masalah': ['Rendahnya aktivitas fisik dan sosial lansia', 'Keterbatasan ruang publik yang aman dan nyaman bagi lansia']
            }
        }
    },
    'Sektor Swasta': {
        'keywords': ['DBS Foundation', 'Bank DBS Indonesia', 'Living Well Seniors Communities', 'Mayapada Hospital', 'PT Blue Bird Tbk'],
        'data': {
            'Bank DBS Indonesia': {
                'fokus': 'Solusi perbankan yang inklusif dan edukasi finansial untuk lansia.',
                'inisiatif': ['Melakukan survei DBS Ageing Society', 'Mengadakan diskusi Impact Beyond Dialogue', 'Mengembangkan solusi perbankan untuk perencanaan pensiun'],
                'fokus_masalah': ['Kesenjangan pengetahuan tentang perencanaan pensiun', 'Rendahnya literasi keuangan di kalangan lansia']
            },
            'PT Blue Bird Tbk': {
                'fokus': 'Penyediaan layanan transportasi yang ramah dan inklusif bagi lansia dan disabilitas.',
                'inisiatif': ['Meluncurkan layanan Lifecare Taxi'],
                'fokus_masalah': ['Keterbatasan akses transportasi yang aman dan nyaman bagi lansia']
            },
            'Living Well Seniors Communities': {
                'fokus': 'Pengembangan infrastruktur dan komunitas perawatan bagi lansia.',
                'inisiatif': ['Berbagi pengalaman dari negara lain', 'Mendorong investasi infrastruktur perawatan'],
                'fokus_masalah': ['Kurangnya ahli gerontologi di Indonesia', 'Keterbatasan investasi pada infrastruktur untuk populasi menua']
            },
            'Mayapada Hospital': {
                'fokus': 'Perawatan kesehatan dan skrining penyakit pada lansia.',
                'inisiatif': ['Memberikan pelayanan kesehatan dan skrining penyakit'],
                'fokus_masalah': ['Kurangnya skrining penyakit rutin', 'Tingginya prevalensi penyakit tidak menular pada lansia']
            }
        }
    },
    'Akademisi & Peneliti': {
        'keywords': ['Universitas Indonesia', 'UI', 'Telkom University', 'Universitas Ahmad Dahlan', 'UAD', 'BRIN', 'Badan Riset dan Inovasi Nasional', 'The SMERU Institute', 'Pakar'],
        'data': {
            'Universitas Indonesia': {
                'fokus': 'Penelitian ilmiah dan pengembangan inovasi di bidang kesehatan geriatri dan mental.',
                'inisiatif': ['Meneliti masalah kesehatan dan mental lansia', 'Mengembangkan aplikasi e-Health Care Pandu-Ina'],
                'fokus_masalah': ['Tingginya risiko depresi dan demensia pada lansia', 'Kurangnya dukungan teknologi untuk caregiver']
            },
            'Telkom University': {
                'fokus': 'Inovasi teknologi berbasis Artificial Intelligence (AI) dan Internet of Things (IoT) untuk keselamatan lansia.',
                'inisiatif': ['Menciptakan inovasi De-Lansia'],
                'fokus_masalah': ['Tingginya risiko jatuh dan cedera serius pada lansia', 'Kurangnya pemantauan aktivitas lansia secara real-time']
            },
            'Universitas Ahmad Dahlan': {
                'fokus': 'Edukasi kesehatan dan pencegahan penyakit pada lansia melalui kolaborasi komunitas.',
                'inisiatif': ['Bekerja sama dengan Social Club Indonesia untuk mencegah demensia', 'Memberikan edukasi tentang kesehatan fisik dan mental'],
                'fokus_masalah': ['Rendahnya pemahaman lansia tentang kesehatan holistik']
            },
            'BRIN': {
                'fokus': 'Analisis kebijakan dan penelitian mendalam mengenai kerentanan sosial dan ekonomi lansia.',
                'inisiatif': ['Meneliti efektivitas program perlindungan sosial'],
                'fokus_masalah': ['Keterbatasan data yang komprehensif', 'Kesenjangan akses lansia terhadap program perlindungan sosial']
            }
        }
    },
    'Organisasi Masyarakat Sipil & Komunitas': {
        'keywords': ['Alzheimer Indonesia', 'Social Club Indonesia', 'SCI', 'Aisyiyah', 'Yayasan Karya Bhakti RIA Pembangunan'],
        'data': {
            'Alzheimer Indonesia': {
                'fokus': 'Pemberdayaan lansia dan edukasi tentang pencegahan demensia.',
                'inisiatif': ['Menekankan pentingnya pemberdayaan lansia agar tidak dianggap sebagai beban'],
                'fokus_masalah': ['Mitos dan stigma negatif yang menganggap lansia sebagai tidak produktif dan tidak berdaya']
            },
            'Aisyiyah': {
                'fokus': 'Penyediaan layanan sosial dan advokasi di tingkat akar rumput.',
                'inisiatif': ['Mengelola panti dan daycare lansia', 'Mendorong kader untuk mengawal implementasi kebijakan kelanjutusiaan'],
                'fokus_masalah': ['Beban perawatan lansia yang mayoritas ditanggung oleh keluarga', 'Keterbatasan lembaga sosial yang memadai']
            },
            'Social Club Indonesia': {
                'fokus': 'Mengadakan kegiatan sosial dan fisik untuk menjaga kesehatan mental dan kognitif lansia.',
                'inisiatif': ['Menyelenggarakan berbagai kegiatan seperti edukasi, senam, dan pengembangan hobi'],
                'fokus_masalah': ['Masalah kesepian dan isolasi sosial yang dapat memicu depresi dan demensia']
            },
            'Yayasan Karya Bhakti RIA Pembangunan': {
                'fokus': 'Pengelolaan panti dan rusun khusus lansia sebagai tempat tinggal dan interaksi sosial.',
                'inisiatif': ['Mengelola rusun lansia yang dirancang untuk kenyamanan dan sosialisasi penghuni'],
                'fokus_masalah': ['Keterbatasan jumlah panti jompo yang memadai dan terjangkau di Indonesia']
            }
        }
    }
}

try:
    df_artikel = pd.read_csv(file_path, sep=';', encoding='latin-1')

    # Dict untuk menyimpan pemangku kepentingan unik dengan detailnya
    unique_stakeholders = {}

    # Iterasi melalui setiap baris DataFrame
    for _, row in df_artikel.iterrows():
        artikel_text = str(row['Artikel'])

        for category, data in stakeholder_data.items():
            for keyword in data['keywords']:
                if re.search(r'\b' + re.escape(keyword) + r'\b', artikel_text, re.IGNORECASE):
                    # Identifikasi nama standar
                    standard_name = keyword
                    for name_variant in data['data'].keys():
                        if name_variant.lower() in keyword.lower():
                            standard_name = name_variant
                            break

                    # Tambahin ke dictionary unik jika belum ada
                    if standard_name not in unique_stakeholders:
                        details = data['data'].get(standard_name, {})

                        fokus_utama = details.get('fokus', 'Tidak disebutkan.')

                        inisiatif_list = details.get('inisiatif', [])
                        inisiatif_str = '\n- '.join(inisiatif_list) if inisiatif_list else 'Tidak disebutkan secara spesifik.'
                        if inisiatif_str != 'Tidak disebutkan secara spesifik.':
                           inisiatif_str = '- ' + inisiatif_str

                        masalah_list = details.get('fokus_masalah', [])
                        masalah_str = '\n- '.join(masalah_list) if masalah_list else 'Tidak disebutkan secara spesifik.'
                        if masalah_str != 'Tidak disebutkan secara spesifik.':
                           masalah_str = '- ' + masalah_str

                        unique_stakeholders[standard_name] = {
                            'Kategori Organisasi': category,
                            'Fokus Utama': fokus_utama,
                            'Rencana/Inisiatif yang Dilakukan': inisiatif_str,
                            'Fokus Masalah yang Ditangani': masalah_str
                        }

    # Ubah dict ke DataFrame
    df_stakeholder = pd.DataFrame.from_dict(unique_stakeholders, orient='index')
    df_stakeholder.index.name = 'Pemangku Kepentingan'
    df_stakeholder.reset_index(inplace=True)


    pd.set_option('display.max_colwidth', None)


    print("Tabel Klasifikasi Pemangku Kepentingan Berdasarkan Analisis Data Anda:\n")
    display(df_stakeholder)

except FileNotFoundError:
    print(f"Error: File not found at {file_path}. Please check the file path and name.")
except Exception as e:
    print(f"An error occurred: {e}")